<a href="https://colab.research.google.com/github/wjayesh/DeGAN/blob/main/DenoiseChannel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/IDKiro/CBDNet-pytorch.git


Cloning into 'CBDNet-pytorch'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 149 (delta 15), reused 44 (delta 13), pack-reused 99
Receiving objects: 100% (149/149), 7.77 MiB | 14.33 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [ ]:
!ls

bin	 dev		    lib    mnt	       root  sys		usr
boot	 etc		    lib32  opt	       run   tensorflow-1.15.2	var
content  home		    lib64  proc        sbin  tmp
datalab  ImageDenoisingGAN  media  python-apt  srv   tools


In [ ]:
!cd ..


In [ ]:
!cd ImageDenoisingGAN/

In [1]:
!git clone https://github.com/manumathewthomas/ImageDenoisingGAN.git

Cloning into 'ImageDenoisingGAN'...
remote: Enumerating objects: 101, done.
remote: Total 101 (delta 0), reused 0 (delta 0), pack-reused 101
Receiving objects: 100% (101/101), 22.47 MiB | 22.91 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [ ]:
/content/drive

In [13]:
!python ImageDenoisingGAN/eval_mse.py

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

(122, 51, 2)




Instructions for updating:
Please use `layer.__call__` method instead.



2022-04-22 13:33:09.878644: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-04-22 13:33:09.892563: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-22 13:33:09.892615: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (29769abbba8a): /proc/driver/nvidia/version does not exist
2022-04-22 13:33:09.892956: I tensorflo

In [4]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3 MB 25 kB/s 
     |████████████████████████████████| 503 kB 47.4 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 3.8 MB 24.5 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=16ed862dbfc9827f405ac732051716e2e4cc942fcb3cfc9f964c91b210a1cf68
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succes

In [ ]:
import numpy as np
from test import *

def get_data(db: int):
  from numpy import genfromtxt
  X_train = genfromtxt(f'/content/drive/MyDrive/btp/data/btp{db}dbnew/zls_train.csv', delimiter=',')
  Y_train = genfromtxt(f'/content/drive/MyDrive/btp/data/btp{db}dbnew/z_train.csv', delimiter=',')

  X_valid = genfromtxt(f'/content/drive/MyDrive/btp/data/btp{db}dbnew/zls_validate.csv', delimiter=',')
  Y_valid = genfromtxt(f'/content/drive/MyDrive/btp/data/btp{db}dbnew/z_validate.csv', delimiter=',')

  X_test = genfromtxt(f'/content/drive/MyDrive/btp/data/btp{db}dbnew/zls_test.csv', delimiter=',')
  Y_test = genfromtxt(f'/content/drive/MyDrive/btp/data/btp{db}dbnew/z_test.csv', delimiter=',')

  X_train = np.reshape(X_train, [8000, 10, 51, 2]);
  Y_train = np.reshape(Y_train, [8000, 10, 51, 2]);

  X_valid = np.reshape(X_valid, [4000, 10, 51, 2]);
  Y_valid = np.reshape(Y_valid, [4000, 10, 51, 2]);

  X_test = np.reshape(X_test, [3000, 10, 51, 2]);
  Y_test = np.reshape(Y_test, [3000, 10, 51, 2]);

  X_train = X_train.astype('float32') / 255;
  Y_train = Y_train.astype('float32') / 255;
  X_valid = X_valid.astype('float32') / 255;
  Y_valid = Y_valid.astype('float32') / 255;
  X_test = X_test.astype('float32') / 255;
  Y_test = Y_test.astype('float32') / 255;

  return X_train, Y_train, X_valid, Y_valid, X_test, Y_test


def calculate_mse(Y_pred, Y_test):
  Y_pred = np.reshape(Y_pred, [10, 51, 2])
  return np.square(np.subtract(Y_pred, Y_test)).mean()

if __name__ == "__main__":
  X_train, Y_train, X_valid, Y_valid, X_test, Y_test = get_data(10)
  mse = 0
  for idx, X in enumerate(X_test):
    Y_pred = denoise(X)
    mse += calculate_mse(Y_pred, Y_test[idx])

  print("Average MSE: ", mse/len(X_test))

In [ ]:
import os
import re
import sys
import glob
import scipy.misc
from itertools import cycle

import numpy as np
import tensorflow as tf


from libs import vgg16

from PIL import Image


LEARNING_RATE = 0.002
BATCH_SIZE = 1
BATCH_SHAPE = [BATCH_SIZE, 122, 51, 2]
SKIP_STEP = 10
N_EPOCHS = 500
CKPT_DIR = './Checkpoints/'
IMG_DIR = './Images/'
GRAPH_DIR = './Graphs/'
TRAINING_SET_DIR= './dataset/training/'
# GROUNDTRUTH_SET_DIR='./dataset/groundtruth/'
VALIDATION_SET_DIR='./dataset/validation/'
METRICS_SET_DIR='./dataset/metrics/'
TRAINING_DIR_LIST = []
ADVERSARIAL_LOSS_FACTOR = 0.5
PIXEL_LOSS_FACTOR = 1.0
STYLE_LOSS_FACTOR = 1.0
SMOOTH_LOSS_FACTOR = 1.0
# metrics_image = scipy.misc.imread(METRICS_SET_DIR+'gt.png', mode='RGB').astype('float32')


def initialize(sess):
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(GRAPH_DIR, sess.graph)

    if not os.path.exists(CKPT_DIR):
        os.makedirs(CKPT_DIR)
    if not os.path.exists(IMG_DIR):
        os.makedirs(IMG_DIR)

    ckpt = tf.train.get_checkpoint_state(os.path.dirname(CKPT_DIR))
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
    return saver

def get_training_dir_list():
    training_list = [d[1] for d in os.walk(TRAINING_SET_DIR)]
    global TRAINING_DIR_LIST
    TRAINING_DIR_LIST = training_list[0]
    return TRAINING_DIR_LIST

def load_next_training_batch():
    batch = next(pool)

    # filelist = sorted(glob.glob(TRAINING_SET_DIR+ batch +'/*.png'), key=alphanum_key)
    # batch = np.array([np.array(scipy.misc.imread(fname, mode='RGB').astype('float32')) for fname in filelist])
    # npad =((0, 0), (56, 56), (0, 0), (0, 0))
    # batch = np.pad(batch, pad_width=npad, mode='constant', constant_values=0)
    return batch

# def load_groundtruth():
#     filelist = sorted(glob.glob(GROUNDTRUTH_SET_DIR + '/*.png'), key=alphanum_key)
#     groundtruth = np.array([np.array(scipy.misc.imread(fname, mode='RGB').astype('float32')) for fname in filelist])
#     # npad = ((0, 0), (56, 56), (0, 0), (0, 0))
#     # groundtruth = np.pad(groundtruth, pad_width=npad, mode='constant', constant_values=0)
#     return groundtruth

def load_validation():
    filelist = sorted(glob.glob(VALIDATION_SET_DIR + '/*.png'), key=alphanum_key)
    validation = np.array([np.array(scipy.misc.imread(fname, mode='RGB').astype('float32')) for fname in filelist])
    npad = ((0, 0), (56, 56), (0, 0), (0, 0))
    validation = np.pad(validation, pad_width=npad, mode='constant', constant_values=0)
    return validation

def training_dataset_init():
    filelist = sorted(glob.glob(TRAINING_SET_DIR + '/*.png'), key=alphanum_key)
    batch = np.array([np.array(scipy.misc.imread(fname, mode='RGB').astype('float32')) for fname in filelist])
    batch = split(batch, BATCH_SIZE)
    training_dir_list = get_training_dir_list()
    global pool
    pool = cycle(batch)
    # return training_dir_list


def imsave(filename, image):
    scipy.misc.imsave(IMG_DIR+filename+'.png', image)

def merge_images(file1, file2):
    """Merge two images into one, displayed side by side
    :param file1: path to first image file
    :param file2: path to second image file
    :return: the merged Image object
    """
    image1 = Image.fromarray(np.uint8(file1))
    image2 = Image.fromarray(np.uint8(file2))

    (width1, height1) = image1.size
    (width2, height2) = image2.size

    result_width = width1 + width2
    result_height = max(height1, height2)

    result = Image.new('RGB', (result_width, result_height))
    result.paste(im=image1, box=(0, 0))
    result.paste(im=image2, box=(width1, 0))
    return result


def tryint(s):
    try:
        return int(s)
    except:
        return s

def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]


def split(arr, size):
    arrs = []
    while len(arr) > size:
        pice = arr[:size]
        arrs.append(pice)
        arr = arr[size:]
    arrs.append(arr)
    return arrs


def lrelu(x, leak=0.2, name='lrelu'):
    with tf.variable_scope(name):
        f1 = 0.5 * (1 + leak)
        f2 = 0.5 * (1 - leak)
        return f1 * x + f2 * abs(x)

def RGB_TO_BGR(img):
    img_channel_swap = img[..., ::-1]
    # img_channel_swap_1 = tf.reverse(img, axis=[-1])
    return img_channel_swap


def get_pixel_loss(target,prediction):
    pixel_difference = target - prediction
    pixel_loss = tf.nn.l2_loss(pixel_difference)
    return pixel_loss

def get_style_layer_vgg16(image):
    net = vgg16.get_vgg_model()
    style_layer = 'conv2_2/conv2_2:0'
    feature_transformed_image = tf.import_graph_def(
        net['graph_def'],
        name='vgg',
        input_map={'images:0': image},return_elements=[style_layer])
    feature_transformed_image = (feature_transformed_image[0])
    return feature_transformed_image

def get_style_loss(target,prediction):
    feature_transformed_target = get_style_layer_vgg16(target)
    feature_transformed_prediction = get_style_layer_vgg16(prediction)
    feature_count = tf.shape(feature_transformed_target)[3]
    style_loss = tf.reduce_sum(tf.square(feature_transformed_target-feature_transformed_prediction))
    style_loss = style_loss/tf.cast(feature_count, tf.float32)
    return style_loss

def get_smooth_loss(image):
    batch_count = tf.shape(image)[0]
    image_height = tf.shape(image)[1]
    image_width = tf.shape(image)[2]

    horizontal_normal = tf.slice(image, [0, 0, 0,0], [batch_count, image_height, image_width-1,3])
    horizontal_one_right = tf.slice(image, [0, 0, 1,0], [batch_count, image_height, image_width-1,3])
    vertical_normal = tf.slice(image, [0, 0, 0,0], [batch_count, image_height-1, image_width,3])
    vertical_one_right = tf.slice(image, [0, 1, 0,0], [batch_count, image_height-1, image_width,3])
    smooth_loss = tf.nn.l2_loss(horizontal_normal-horizontal_one_right)+tf.nn.l2_loss(vertical_normal-vertical_one_right)
    return smooth_loss



In [ ]:
import time

import tensorflow as tf
import numpy as np

from utils import *
from model import *

from skimage import measure



def test(image):
    tf.reset_default_graph()

    global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

    gen_in = tf.placeholder(shape=[None, BATCH_SHAPE[1], BATCH_SHAPE[2], BATCH_SHAPE[3]], dtype=tf.float32, name='generated_image')
    real_in = tf.placeholder(shape=[None, BATCH_SHAPE[1], BATCH_SHAPE[2], BATCH_SHAPE[3]], dtype=tf.float32, name='groundtruth_image')

    Gz = generator(gen_in)



    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)

        saver = initialize(sess)
        initial_step = global_step.eval()

        start_time = time.time()
        n_batches = 200
        total_iteration = n_batches * N_EPOCHS

        image = sess.run(tf.map_fn(lambda img: tf.image.per_image_standardization(img), image))
        image = sess.run(Gz, feed_dict={gen_in: image})
        image = np.resize(image[0][56:, :, :], [144, 256, 3])
        # imsave('output', image)
        return image

def denoise(image):
    # image = scipy.misc.imread(image, mode='RGB').astype('float32')
    npad = ((56, 56), (0, 0), (0, 0))
    image = np.pad(image, pad_width=npad, mode='constant', constant_values=0)
    image = np.expand_dims(image, axis=0)
    print(image[0].shape)
    output = test(image)
    return output



if __name__=='__main__':
    image = scipy.misc.imread(sys.argv[-1], mode='RGB').astype('float32')
    npad = ((56, 56), (0, 0), (0, 0))
    image = np.pad(image, pad_width=npad, mode='constant', constant_values=0)
    image = np.expand_dims(image, axis=0)
    print(image[0].shape)
    test(image)


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim


from utils import *
from conv_helper import *


def generator(input):
    conv1, conv1_weights = conv_layer(input, 9, 2, 32, 1, "g_conv1")
    conv2, conv2_weights = conv_layer(conv1, 3, 32, 61, 1, "g_conv2")
    # conv3, conv3_weights = conv_layer(conv2, 3, 64, 128, 1, "g_conv3")

    res1, res1_weights = residual_layer(conv2, 3, 61, 61, 1, "g_res1")
    res2, res2_weights = residual_layer(res1, 3, 61, 61, 1, "g_res2")
    res3, res3_weights = residual_layer(res2, 3, 61, 61, 1, "g_res3")

    deconv1 = deconvolution_layer(res3, [BATCH_SIZE, 61, 61, 61], 'g_deconv1')
    deconv2 = deconvolution_layer(deconv1, [BATCH_SIZE, 122, 51, 32], "g_deconv2")

    deconv2 = deconv2 + conv1

    conv4, conv4_weights = conv_layer(deconv2, 9, 32, 2, 1, "g_conv5", activation_function=tf.nn.tanh)

    conv4 = conv4 + input
    # output = output_between_zero_and_one(conv4)

    return conv4

def discriminator(input, reuse=False):
    conv1, conv1_weights = conv_layer(input, 4, 3, 48, 2, "d_conv1", reuse=reuse)
    conv2, conv2_weights = conv_layer(conv1, 4, 48, 96, 2, "d_conv2", reuse=reuse)
    conv3, conv3_weights = conv_layer(conv2, 4, 96, 192, 2, "d_conv3", reuse=reuse)
    conv4, conv4_weights = conv_layer(conv3, 4, 192, 384, 1, "d_conv4", reuse=reuse)
    conv5, conv5_weights = conv_layer(conv4, 4, 384, 1, 1, "d_conv5", activation_function=tf.nn.sigmoid, reuse=reuse)

    return conv5


In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim as slim

from utils import *

def conv_layer(input_image, ksize, in_channels, out_channels, stride, scope_name, activation_function=lrelu, reuse=False):
    with tf.variable_scope(scope_name):
        filter = tf.Variable(tf.random_normal([ksize, ksize, in_channels, out_channels], stddev=0.03))
        output = tf.nn.conv2d(input_image, filter, strides=[1, stride, stride, 1], padding='SAME')
        output = slim.batch_norm(output)
        if activation_function:
            output = activation_function(output)
        return output, filter

def residual_layer(input_image, ksize, in_channels, out_channels, stride, scope_name):
    with tf.variable_scope(scope_name):
        output, filter = conv_layer(input_image, ksize, in_channels, out_channels, stride, scope_name+"_conv1")
        output, filter = conv_layer(output, ksize, out_channels, out_channels, stride, scope_name+"_conv2")
        output = tf.add(output, tf.identity(input_image))
        return output, filter

def transpose_deconvolution_layer(input_tensor, used_weights, new_shape, stride, scope_name):
    with tf.varaible_scope(scope_name):
        output = tf.nn.conv2d_transpose(input_tensor, used_weights, output_shape=new_shape, strides=[1, stride, stride, 1], padding='SAME')
        output = tf.nn.relu(output)
        return output

def resize_deconvolution_layer(input_tensor, new_shape, scope_name):
    with tf.variable_scope(scope_name):
        output = tf.image.resize_images(input_tensor, (new_shape[1], new_shape[2]), method=1)
        output, unused_weights = conv_layer(output, 3, 61, new_shape[3], 1, scope_name+"_deconv")
        return output

def deconvolution_layer(input_tensor, new_shape, scope_name):
    return resize_deconvolution_layer(input_tensor, new_shape, scope_name)

def output_between_zero_and_one(output):
    output +=1
    return output/2
